In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp preprocessor

# PreProcessor

> An API to Preprocess train, valid and test dataset for Machine Learning models based on tabular or strucuture data

In [3]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [4]:
# export
from tabular_ml_toolkit.dataframeloader import *
from tabular_ml_toolkit.logger import *

In [5]:
# export
# hide
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

In [6]:
# export

class PreProcessor:
    """
    Represent PreProcessor class
    
    Attributes:
    numerical_transformer: Numerical Columns Tranformer
    categorical_transformer: Categorical Columns Transformer
    preprocessor: Preprocessor for Columns Tranformer
    """

    def __init__(self):
        self.numerical_transformer = None
        self.categorical_transformer = None
        self.columns_transfomer = None
        self.transformer_type = None
        self.target_cols_transformer = None
        self.target_cols_pl = None
        self.categorical_cols_pl = None
        self.numerical_cols_pl = None
        
    
    def __str__(self):
        """Returns human readable string reprsentation"""
        attr_str = "numerical_transformer, categorical_transformer,columns_transfomer"
        return "PreProcessor object with attributes:" + attr_str
    
    def __repr__(self):
        return self.__str__()
    
    # PreProcessor core methods
    
    # Preprocessing for numerical data
    def preprocess_numerical_data(self, num_cols__imputer, num_cols__scaler):
        self.numerical_cols_pl = Pipeline(steps=[
            ('imputer', num_cols__imputer),
            ('scaler',  num_cols__scaler)
        ])
        return self.numerical_cols_pl
        
    # Preprocessing for categorical data
    def preprocess_categorical_data(self, cat_cols__imputer, cat_cols__encoder):
        self.categorical_cols_pl = Pipeline(steps=[
        ('imputer', cat_cols__imputer),
        ('encoder', cat_cols__encoder)
        ])
        return self.categorical_cols_pl
    
    # Preprocessing for target cols
    def preprocess_target_cols(self, target_cols__encoder):
        self.target_cols_pl = Pipeline(steps=[
        ('encoder', target_cols__encoder)
        ])
        return self.target_cols_pl
    
    
    # TODO PreProcess for target column in case of Binary and Multi Class Classification
    def encode_targets(self, dataframeloader, target_cols__encoder):
        # create scikit-learn pipelines instances
        self.target_cols_pl = self.preprocess_target_cols(target_cols__encoder)

        # convert all categorical columns to LabelEncoder with Scikit-learn ColumnTranfomer
        self.target_cols_transfomer = ColumnTransformer(
            transformers=[
                ('target_cols', self.target_cols_transformer,
                 dataframeloader.y)
            ])
        
        return self.target_cols_transformer
    
    # Bundle preprocessing pipelines based upon types of columns
    def preprocess_all_cols(self, dataframeloader, problem_type="regression",
                            num_cols__imputer=SimpleImputer(strategy='median'),
                            num_cols__scaler=StandardScaler(),
                            cat_cols__imputer=SimpleImputer(strategy='constant'),
                            cat_cols__encoder=OneHotEncoder(handle_unknown='ignore'),
                            target_cols__encoder=LabelEncoder()):
#                             cat_cols__encoder=OrdinalEncoder(handle_unknown='use_encoded_value',
#                                                             unknown_value=np.nan)):
        
        # encode target based upon problem type
        if problem_type == "classification":
            logger.info("PreProcessing will include target(s) encoding!")
            self.encode_targets(dataframeloader=dataframeloader, target_cols__encoder=target_cols__encoder)
            
        
        # change preprocessor according to type of column found
        
        if len(dataframeloader.categorical_cols) < 1:
            logger.info("categorical columns are None, Preprocessing will done accordingly!")
            # create scikit-learn pipelines instances
            self.numerical_cols_pl = self.preprocess_numerical_data(num_cols__imputer, num_cols__scaler)
            
            # scaled and impute all numerical columns
            self.columns_transfomer = ColumnTransformer(
                transformers=[
                    ('num_cols', self.numerical_cols_pl,
                     dataframeloader.numerical_cols)
                    ##TODO add here target columns tranformer
                ])
            
        elif len(dataframeloader.numerical_cols) < 1:
            logger.info("numerical columns are None, Preprocessing will done accordingly!")
            # create scikit-learn pipelines instances
            self.categorical_transformer = self.preprocess_categorical_data(cat_cols__imputer, cat_cols__encoder)
            
            # convert all categorical columns to OneHotEncoder with Scikit-learn ColumnTranfomer
            self.columns_transfomer = ColumnTransformer(
                transformers=[
                    ('cat_cols', self.categorical_cols_pl,
                     dataframeloader.categorical_cols)
                ])
        
        else:
            # create scikit-learn pipelines instances
            logger.info("Both Numerical & Categorical columns found, Preprocessing will done accordingly!")
            self.numerical_transformer = self.preprocess_numerical_data(num_cols__imputer, num_cols__scaler)
            self.categorical_transformer = self.preprocess_categorical_data(cat_cols__imputer, cat_cols__encoder)
            
            # convert to Scikit-learn ColumnTranfomer
            self.columns_transfomer = ColumnTransformer(
                transformers=[
                    ('num_cols', self.numerical_cols_pl,
                     dataframeloader.numerical_cols),
                    ('cat_cols', self.categorical_cols_pl,
                     dataframeloader.categorical_cols)
                ])
            
        #set tranformer type
        self.transformer_type = self.columns_transfomer
        
        return self

In [7]:
# showdoc(PreProcessor.preprocess_data)

#### Test PreProcessor with House Data

In [8]:
dfl = DataFrameLoader().from_csv(
    train_file_path="input/home_data/train.csv",
    test_file_path="input/home_data/test.csv",
    idx_col="Id", target="SalePrice",
    random_state=42)
dfl

2021-11-18 23:54:02,521 INFO DataFrame Memory usage decreased to 0.58 Mb (35.5% reduction)
2021-11-18 23:54:02,559 INFO DataFrame Memory usage decreased to 0.58 Mb (34.8% reduction)


DataFrameLoader object with attributes: X_full, X_test, X(features), y(target), X_train, X_valid, y_train and y_valid

In [9]:
# let's create train, valid split
dfl.create_train_valid(valid_size=0.2, random_state = 42)

In [10]:
pp = PreProcessor().preprocess_all_cols(dataframeloader=dfl, problem_type="regression")
pp

2021-11-18 23:54:02,645 INFO Both Numerical & Categorical columns found, Preprocessing will done accordingly!


PreProcessor object with attributes:numerical_transformer, categorical_transformer,columns_transfomer

In [11]:
# pp.columns_transfomer

In [12]:
print(len(dfl.numerical_cols))
# dfl.numerical_cols

36


In [13]:
print(len(dfl.high_card_cat_cols))
# dfl.high_card_cat_cols

3


In [14]:
print(len(dfl.low_card_cat_cols))
# dfl.low_card_cat_cols

40


In [15]:
# Define model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model

RandomForestRegressor(random_state=42)

In [16]:
# Bundle preprocessor and model in a pipeline
pl = Pipeline(steps=[('preprocessor', pp.columns_transfomer),
                      ('model', model)
                     ])
# pl

In [17]:
# for visualizing pipeline
from sklearn import set_config

set_config(display="diagram")
pl

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Lo...
                                                   'LotConfig', 'LandSlope',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'MasVnrType', 'ExterQual',
                                                   'ExterCond', 'Foundation',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...])])),
                ('model', RandomForestRegressor(random_state=42))])

In [18]:
# Preprocessing of training data and then fit model 
pl.fit(dfl.X_train, dfl.y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCond', 'YearBuilt',
                                                   'YearRemodAdd', 'MasVnrArea',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'Lo...
                                                   'LotConfig', 'LandSlope',
                                                   'Condition1', 'Condition2',
                                                   'BldgType', 'HouseStyle',
                                                   'RoofStyle', 'RoofMatl',
                                                   'MasVnrType', 'ExterQual',
                                                   'ExterCond', 'Foundation',
                                                   'BsmtQual', 'BsmtCond',
                                                   'BsmtExposure',
                                                   'BsmtFinType1',
                                                   'BsmtFinType2', 'Heating',
                                                   'HeatingQC', 'CentralAir',
                                                   'Electrical', 'KitchenQual',
                                                   'Functional', 'FireplaceQu', ...])])),
                ('model', RandomForestRegressor(random_state=42))])

In [19]:
# Preprocessing of validation data and then get predictions
preds = pl.predict(dfl.X_valid)

print('X_valid MAE:', mean_absolute_error(dfl.y_valid, preds))

X_valid MAE: 17634.989965753426


In [20]:
# X_valid MAE: 17582.46150684932

In [21]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_dataframeloader.ipynb.
Converted 01_preprocessor.ipynb.
Converted 02_tmlt.ipynb.
Converted 03_Tutorial.ipynb.
Converted 04_optuna_objective.ipynb.
Converted 07_Kaggle_TPS_Tutorial.ipynb.
Converted do_optuna_opt_tutorial.ipynb.
Converted index.ipynb.
Converted logger.ipynb.
